# A blogging workflow with Jupyter and Wordpress

Abstract: How to convert your Jupyter notebook as blog post, including all the Wordpress features, plots, custom formatting, and latex formulas.

For me, [Jyputer](http://jupyter.org/) is a ready made solution for two problems simultaneously. It's a great tool for prototyping all sorts of algorithms in the [SciPy](https://www.scipy.org/) environment, while it also does fantastic job in taking up notes on the mathematics behind those algorithms thanks to Markdown and the beautiful formatting of mathematical formulas using the Latex syntax.

To publish a Jupyter notebook, you can simply upload the .ipynb to Github, which renders as a static page as you open it there. How

$\nabla \times \mathbf{E} = 0$


In [1]:
# preample
import numpy as np
import matplotlib.pyplot as plt; 
from IPython.display import display
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')